# Course 1: Lidar
## Part 2: Point Cloud Segmentation 
#### By Jonathan L. Moran (jonathan.moran107@gmail.com)
From the Sensor Fusion Nanodegree programme offered at Udacity.

## Objectives

* Use the [Point Cloud Library](https://pointclouds.org/) (PCL) to segment point clouds;
* Implement the [Random Sample Consensus](https://en.wikipedia.org/wiki/Random_sample_consensus) (RANSAC) algorithm for planar model fitting;
* Perform ground plane separation to "remove" the road surface from our point cloud data.

## 1. Introduction

### 1.1. Point Cloud Segmentation

Our goal in this lesson is to _process_ the point cloud data we received from the LiDAR sensor and _segment_ it in such a way that we are able to "remove" any of the points from our scene which do not belong to the obstacles we wish to detect. In the driving domain, one common type of points in which we wish to remove are _ground plane_ points. These are the points which "belong" to the road surface(s) that the vehicle(s) travel along. 

There are many potential use cases for LiDAR sensors in the automotive space, with point cloud segmentation being one such use case discussed in detail here. There are many other uses for LiDAR sensors and point clouds in the automotive and robotics space: multi-modal sensor calibration / fusion techniques, semi-supervised deep learning models for depth estimation, and for bench-marking tasks used to e.g., evaluate the performance of a "stand-alone" sensor on a given task (such as RGB camera-based vision systems on the monocular depth estimation task).

In the following section, we will narrow our scope to focus only on ground plane estimation as it relates to point cloud segmentation in the section below...

## 2. Programming Task

## 3. Closing Remarks

#### Alternatives


#### Extensions of task

## 4. Future Work

* ⬜️ ;
* ⬜️ .

## Credits

This assignment was prepared by Aaron Brown and Michael Maile of Mercedes-Benz Research & Development of North America (MBRDNA), 2021 (link [here](https://learn.udacity.com/nanodegrees/nd313/)).


References
* [1]

Helpful resources:
* 